In [4]:
import warnings
warnings.filterwarnings("ignore")
import sys, os, re, csv, codecs, numpy as np, pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation,GRU,CuDNNLSTM,CuDNNGRU
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
import gc

In [ ]:
!wget https://s3-us-west-1.amazonaws.com/fasttext-vectors/crawl-300d-2M.vec.zip

In [ ]:
!unzip crawl-300d-2M.vec.zip

In [ ]:
#!wget http://nlp.stanford.edu/data/glove.6B.zip

In [5]:
embed_size = 300 # how big is each word vector
max_features = 200000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 1000 # max number of words in a comment to use


In [6]:

EMBEDDING_FILE='./crawl-300d-2M.vec'
TRAIN_DATA_FILE='./train.csv'
TEST_DATA_FILE='./test.csv'



In [7]:


train = pd.read_csv(TRAIN_DATA_FILE)
test = pd.read_csv(TEST_DATA_FILE)

list_sentences_train = train["comment_text"].fillna("_na_").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("_na_").values



In [8]:
%%time
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [9]:
gc.collect()

0

In [10]:
%%time
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE))

In [11]:
%%time
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [17]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(CuDNNLSTM(40, return_sequences=True))(x)
x=  Bidirectional(CuDNNGRU(40, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.3)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
model.fit(X_t, y, batch_size=512, epochs=2)

Epoch 1/2
159571/159571 [==============================] - 357s 2ms/step - loss: 0.1177 - acc: 0.9654
Epoch 2/2
159571/159571 [==============================] - 328s 2ms/step - loss: 0.0457 - acc: 0.9829


In [19]:
model.fit(X_t, y, batch_size=512, epochs=2);

Epoch 1/2
159571/159571 [==============================] - 329s 2ms/step - loss: 0.0390 - acc: 0.9850
Epoch 2/2
159571/159571 [==============================] - 328s 2ms/step - loss: 0.0340 - acc: 0.9866


In [22]:
model.fit(X_t, y, batch_size=512, epochs=2);

Epoch 1/2
159571/159571 [==============================] - 328s 2ms/step - loss: 0.0294 - acc: 0.9884
Epoch 2/2
159571/159571 [==============================] - 328s 2ms/step - loss: 0.0254 - acc: 0.9901


In [25]:
model.fit(X_t, y, batch_size=512, epochs=3);

Epoch 1/3
159571/159571 [==============================] - 328s 2ms/step - loss: 0.0222 - acc: 0.9915
Epoch 2/3
159571/159571 [==============================] - 328s 2ms/step - loss: 0.0194 - acc: 0.9926
Epoch 3/3
159571/159571 [==============================] - 327s 2ms/step - loss: 0.0168 - acc: 0.9938


In [28]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [26]:
y_test = model.predict([X_te], batch_size=1024, verbose=1)
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission[list_classes] = y_test
sample_submission.to_csv('submission-lstm-gru.csv', index=False)


153164/153164 [==============================] - 86s 562us/step


In [27]:
gc.collect()

893